Codice che genera tutte le configurazioni possibili di griglie 6x6 con numero di buchi da 1 a 9 con la condizione che due buchi non possono essere 
disposti in due siti vicini (considerando anche le PBC): ciascun buco "blocca" i suoi 6 primi vicini, perché se c'è un buco, i primi vicini devono essere occupati.
9 è il numero massimo di buchi che posso avere affinché vengano rispettate le regole di selezione (incluse PBC): un esempio è la configurazione a "massimo impacchettamento" con esattamente un atomo tra ogni buco

In [1]:
import os
import numpy as np
from itertools import combinations

L = 6  # dimensione della griglia: 6x6 triangoli
primi_vicini = [(-1, -1), (0, -1), (1, 0), (-1, 0), (0, 1), (1, 1)]
#lista che contiene le posizioni dei primi vicini relative al sito di riferimento


def modulo(i):
    """
    Funzione modulo: usata nel calcolo dei primi vicini per tenere conto delle PBC
    Input:
        i (intero): indice di posizione, assume valori nell'intervallo [-1,6]
    Parametri:
        L (intero): dimensione della griglia
    Returns:
        intero: sostituisce i valori di input i=-1 e i=6 con 6 -> 0 e -1 -> 5
    """
    return i % L

# 
def ha_vicini(combinazione):
    """
    Verifica se nella combinazione in input ci sono buchi vicini (inclusi vicini con PBC)
    Input:
        combinazione (tupla): contiene le coordinate (i,j) dei buchi
    Returns:
        bool: restituisce True se trova due buchi vicini, False se non li trova
    """
    #per ciascun buco, controllo se uno o più dei suoi primi vicini si trova nella combinazione
    for i, j in combinazione:
        for di, dj in primi_vicini:
            ni = modulo(i + di)
            nj = modulo(j + dj)
            if (ni, nj) in combinazione:
                return True
    return False
    
def genera_configurazioni(n_buchi, cartella_output):
    """
    Generare configurazioni valide per un dato n_buchi
    Input:
        N-buchi (intero): numero di vacanze (da 1 a 9)
        output_dir (stringa): percorso della cartella dove salvare le configurazioni
    Returns:
        tot_config (intero): numero totale di configurazioni permesse con N_buchi
    """
    #genero percorso cartella se non esiste già
    os.makedirs(cartella_output, exist_ok=True)
    #costruisco matrice 6x6 di 1 che rappresentano tutti i siti occupati della griglia 6x6
    griglia = [(i, j) for i in range(L) for j in range(L)]
    tot_config = 0

    #genero tutte le combinazioni di n_buchi e controllo quali di queste sono valide
    for combinazione in combinations(griglia, n_buchi):
        if ha_vicini(combinazione):
            continue
        #se la configurazione è valida, le entrate che corrispondono alle posizioni dei buchi vengono trasformate in 0, ossia siti vuoti della griglia
        matrice = np.ones((L, L), dtype=int)
        for (i, j) in combinazione:
            matrice[i, j] = 0

        #nella cartella {n}_vacancies creo un file per ciascuna configurazione ("B coord_x coord_y coord_z")
        #se n_buchi=2 e il file contiene la configurazione con buchi in (1,1) e (3,4), il file viene denominato "(1,1)_(3,4).dat"
        nomefile = "_".join(f"({i},{j})" for (i, j) in combinazione)
        filepath = os.path.join(cartella_output, f"{nomefile}.dat")
        with open(filepath, 'w') as f:
            for x in range(L):
                for y in range(L):
                    #se le entrate (x,y) della matrice sono 1 (sito pieno): (coord_x, coord_y, coord_z) = (1/6*x, 1/6*y, 0.400000)
                    #se le entrate (x,y) della matrice sono 0 (sito vuoto): non scrivo niente
                    if matrice[x, y] == 1:
                        f.write(f"B  {x / L:.6f}  {y / L:.6f}  0.400000\n")
        #conteggio le configurazioni
        tot_config += 1

    print(f"{tot_config} configurazioni permesse trovate con n_buchi = {n_buchi} in '{cartella_output}'.")
    return tot_config

#Genero una cartella contenente le configurazioni per ciascun numero di buchi (da 1 a 9)
for n_buchi in range(1, 3):
    cartella = f"./allowed_config/{n_buchi}_vacancies"
    genera_configurazioni(n_buchi, cartella)


36 configurazioni permesse trovate con n_buchi = 1 in './allowed_config/1_vacancies'.
522 configurazioni permesse trovate con n_buchi = 2 in './allowed_config/2_vacancies'.


Script per generare i file relax.in per eseguire i rilassamenti delle configurazioni di borofene generate su 4 layers di substrato di Re(0001).
La cella unitaria contiene 48 atomi di Re (12 per ogni layer) e un numero variabile di atomi di B (al mssimo 35 per configurazioni con un buco solo). In totale la unit cell contiene al massimo 83 atomi.
Fisso le posizioni degli atomi di Re dei layers più interni, mentre i due layers superficiali sono liberi di rilassarsi. La distanza tra i layers di re è c/2 = 4.458/2 = 2.229 ang e uso questa distanza anche come gap iniziale tra borofene e ultimo strato di boro.
Tra una cella e la successiva lungo z rimane un vuoto di 13 ang che dovrebbe essere sufficiente a garantire l'isolamento.
La distanza tra gli atomi di B per avere il corretto allineamento e periodicità layer-substrato è 1.594 ang.
Per determinare i parametri faccio un test di convergenza sulla struttura triangolare? Servirebbe una cella unitaria 3x3 con 21 atomi (12 di Re e 9 di B).

In [2]:
import os

def leggi_dat(path):
    """
    Legge le configurazioni del borofene nei file .dat
    Input:
        path (stringa): percorso dei file
    Returns:
        f.readlines() (lista): contiene le stringhe con le coordinate cristallografiche di ciascun atomo di B
    """
    with open(path, 'r') as f:
        return f.readlines()


def genera_relax_in(B_coordinates):
    """
    Crea i file relax.in per ciascuna configurazione di borofene su substrato di Re(0001) con 4 layers
    Input:
        B_coordinates (lista): coordinate degli atomi di B
        nat (intero): numero totale di atomi (48 di Re + B variabile tra 27 e 35)
    Returns:
        testo da inserire in relax.in
    """
    nat = 48 + len(B_coordinates)
    return f"""&control
    calculation = 'relax',
    prefix = 'borophene',
    outdir = './outdir',
    pseudo_dir = '../../../pseudo'
    /
&system
    ibrav = 4,
    celldm(1) = 18.074, !2*sqrt(3)*2.761 = 9.564 ang = 18.074 bohr
    celldm(3) = 2.3304, !4.458*5/9.564 con vuoto = 4.458*3 = 13.374 A
    nat = {nat},
    ntyp = 2,
    ecutwfc = 60,
    occupations= 'smearing',
    smearing   = 'marzari-vanderbilt',
    degauss = 0.03
    /
&electrons
    /
&ions
    /
ATOMIC_SPECIES
 B  10.811  B.pbesol-n-rrkjus_psl.1.0.0.UPF
 Re 186.207 re_pbesol_v1.2.uspp.F.UPF
ATOMIC_POSITIONS crystal
Re 0.166667 0.000000 0.000000 0 0 0
Re 0.666667 0.000000 0.000000 0 0 0
Re 0.000000 0.166667 0.000000 0 0 0
Re 0.500000 0.166667 0.000000 0 0 0
Re 0.333333 0.333333 0.000000 0 0 0
Re 0.833333 0.333333 0.000000 0 0 0
Re 0.166667 0.500000 0.000000 0 0 0
Re 0.666667 0.500000 0.000000 0 0 0
Re 0.000000 0.666667 0.000000 0 0 0
Re 0.500000 0.666667 0.000000 0 0 0
Re 0.333333 0.833333 0.000000 0 0 0
Re 0.833333 0.833333 0.000000 0 0 0
Re 0.000000 0.000000 0.100000 0 0 0
Re 0.500000 0.000000 0.100000 0 0 0
Re 0.333333 0.166667 0.100000 0 0 0
Re 0.833333 0.166667 0.100000 0 0 0
Re 0.166667 0.333333 0.100000 0 0 0
Re 0.666667 0.333333 0.100000 0 0 0
Re 0.000000 0.500000 0.100000 0 0 0
Re 0.500000 0.500000 0.100000 0 0 0
Re 0.333333 0.666667 0.100000 0 0 0
Re 0.833333 0.666667 0.100000 0 0 0
Re 0.166667 0.833333 0.100000 0 0 0
Re 0.666667 0.833333 0.100000 0 0 0
Re 0.166667 0.000000 0.200000
Re 0.666667 0.000000 0.200000
Re 0.000000 0.166667 0.200000
Re 0.500000 0.166667 0.200000
Re 0.333333 0.333333 0.200000
Re 0.833333 0.333333 0.200000
Re 0.166667 0.500000 0.200000
Re 0.666667 0.500000 0.200000
Re 0.000000 0.666667 0.200000
Re 0.500000 0.666667 0.200000
Re 0.333333 0.833333 0.200000
Re 0.833333 0.833333 0.200000
Re 0.000000 0.000000 0.300000
Re 0.500000 0.000000 0.300000
Re 0.333333 0.166667 0.300000
Re 0.833333 0.166667 0.300000
Re 0.166667 0.333333 0.300000
Re 0.666667 0.333333 0.300000
Re 0.000000 0.500000 0.300000
Re 0.500000 0.500000 0.300000
Re 0.333333 0.666667 0.300000
Re 0.833333 0.666667 0.300000
Re 0.166667 0.833333 0.300000
Re 0.666667 0.833333 0.300000
{''.join(B_coordinates)}
K_POINTS automatic
6 6 1 0 0 0
"""

for n_buchi in range(1,3):
    dat_folder = os.path.join("allowed_config", f"{n_buchi}_vacancies")
    qe_folder = os.path.join("qe_jobs", f"{n_buchi}_vacancies")

    #Evita errori se mancano cartelle in allowed_config
    if not os.path.isdir(dat_folder):
        continue

    os.makedirs(qe_folder, exist_ok=True)

    for dat_file in os.listdir(dat_folder):
        if not dat_file.endswith(".dat"):
            continue

        name = dat_file.replace(".dat", "")
        config_dir = os.path.join(qe_folder, name)
        os.makedirs(config_dir, exist_ok=True)

        # Creo cartella outdir
        outdir_path = os.path.join(config_dir, "outdir")
        os.makedirs(outdir_path, exist_ok=True)

        # Leggo le coordinate e creo relax.in
        dat_path = os.path.join(dat_folder, dat_file)
        B_coordinate = leggi_dat(dat_path)
        relax_in = genera_relax_in(B_coordinate)

        relax_path = os.path.join(config_dir, f"{name}.relax.in")
        with open(relax_path, 'w') as f:
            f.write(relax_in)

Organizzazione cartelle: per ciascuna configurazione genero una cartella denominata con i buchi della configurazione "(-)_..._(-)" che contiene il file di input "(-)_..._(-).relax.in" e la directory per i files di output "outdir". I due PPs sono comuni e si trovano nella cartella pseudo
- Bo_config
    - allowed_config
        - 1_vacancies (contiene i files .dat con un buco)
          
         ...
      
        - 9_vacancies (contiene i files .dat con nove buchi)
    - qe_jobs
        - 1_vacancies
            - (-)_..._(-)
                - (-)_..._(-).relax.in
                - outdir
                  
            ...
          
            - (-)_..._(-)
                - (-)_..._(-).relax.in
                - outdir
         ...
        - 9_vacancies
            - (-)_..._(-)
                - (-)_..._(-).relax.in
                - outdir
                  
            ...
          
            - (-)_..._(-)
                - (-)_..._(-).relax.in
                - outdir
    - pseudo
        - B.pbesol-n-rrkjus_psl.1.0.0.UPF
        - re_pbesol_v1.2.uspp.F.UPF
        

Per eseguire le simulazioni con pw.x -inp (-)...(-).relax.in > (-)...(-).relax.out
La simulazioni vanno eseguito nella dir "Bo_config"

In [ ]:
import subprocess

for n_buchi in range(1, 10):
    qe_folder = os.path.join("qe_jobs", f"{n_buchi}_vacancies")

    for config_name in os.listdir(qe_folder):
        config_dir = os.path.join(qe_folder, config_name)

        # Cerco il file .relax.in nella cartella
        for file in os.listdir(config_dir):
            if file.endswith(".relax.in"):
                input_path = os.path.join(config_dir, file)
                output_path = input_path.replace(".relax.in", ".relax.out")
                
                with open(output_path, "w") as out_file:
                    subprocess.run(
                        ["pw.x", "-inp", input_path],
                        stdout=out_file,
                        stderr=subprocess.STDOUT,
                        cwd=config_dir
                    )